In [1]:
%load_ext autoreload
%autoreload 2

import math
import pandas as pd
import seaborn as sns
import warnings
%matplotlib inline
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)
import numpy as np
import pandas as pd
import matplotlib.transforms as transforms


from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import datetime
import statsmodels

# Local Modules
import sys
import os.path
sys.path.insert(0, os.path.abspath('/scratch/horta/coronawiki/'))
from helpers.load import load_interventions, load_aggregated, load_topics
from helpers.vars import codes, helper_langs, interventions_helper, topics, codes
from helpers.plot import set_size, plot_dates, plot_interventions
from helpers.topics import order_topics, colors_z, get_df_pageviews_topics
from helpers.diffs_n_diffs import get_standard_error_sum
from dateutil.relativedelta import relativedelta

In [2]:
# Latex Configuration
import matplotlib.font_manager as font_manager
from matplotlib.lines import Line2D
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import os
# fontpath = os.path.expanduser('~/LinLibertine_DRah.ttf')
# prop = font_manager.FontProperties(fname=fontpath)

params = {
    "axes.titlesize" : 14,
    'axes.labelsize': 12,
    'font.size': 12,
    'legend.fontsize': 12,
    'xtick.labelsize': 13,
    'ytick.labelsize': 13,
#     'font.family': 'Linux Libertine Display',
    'text.usetex': True
}

mpl.rcParams.update(params)

## Loads Data

In [3]:
interventions = load_interventions("../data/interventions.csv")
topics_df, topics = load_topics('../data/topics_linked.csv')
agg = load_aggregated("../data/aggregated_timeseries.json.gz")
codes_order = ["sr", "it", "fr", "en", "nl", "ko", "de", "fi", "ja", "no", "sv", "da"]
time_int = 35

In [6]:
topics_df.columns

Index(['index', 'Geography.Regions.Asia.Central Asia',
       'Geography.Regions.Europe.Eastern Europe',
       'History and Society.Military and warfare', 'Culture.Media.Television',
       'History and Society.Education', 'Culture.Media.Books',
       'Geography.Regions.Africa.Africa*', 'Culture.Visual arts.Architecture',
       'Culture.Biography.Women', 'Geography.Regions.Americas.North America',
       'STEM.STEM*', 'Culture.Performing arts', 'Culture.Literature',
       'History and Society.Politics and government',
       'Geography.Regions.Africa.Northern Africa',
       'STEM.Earth and environment', 'Geography.Regions.Africa.Central Africa',
       'STEM.Medicine & Health', 'STEM.Libraries & Information',
       'Geography.Regions.Europe.Europe*',
       'Geography.Regions.Europe.Western Europe', 'Culture.Media.Radio',
       'STEM.Space', 'Culture.Visual arts.Comics and Anime', 'STEM.Technology',
       'Culture.Internet culture', 'Culture.Media.Software',
       'Geography.R

In [ ]:
'Culture.Media.Video games'
'Culture.Media.Books'
'Culture.Media.Television'
'Culture.Media.Radio'
'Culture.Visual arts.Comics and Anime'
'Culture.Internet culture'
'Culture.Media.Software'
'Culture.Food and drink'
'Culture.Visual arts.Fashion'
'Culture.Media.Films'
'Culture.Media.Media*'
'Culture.Philosophy and religion'
'Culture.Sports'
'Culture.Media.Entertainment'
'Culture.Media.Music'

## Performs diffs-in-diffs

In [7]:
mobility = lambda x, delta: interventions[x.replace(".m", "")]["Mobility"] + delta
normalcy = lambda x, delta: interventions[x.replace(".m", "")]["Normalcy"] + delta

df_pageviews, df_pageviews_results = get_df_pageviews_topics(agg, 
                                                      mobility, relativedelta(days=0),
                                                      mobility, relativedelta(days=0),
                                                      time_int)

df_pageviews_norm, df_pageviews_norm_results = get_df_pageviews_topics(agg, 
                                                                mobility, relativedelta(days=time_int), 
                                                                normalcy, relativedelta(days=0), 
                                                                time_int)

df_pageviews_norm2, df_pageviews_norm_results2 = get_df_pageviews_topics(agg, 
                                                            mobility, relativedelta(days=0),
                                                            normalcy, relativedelta(days=0),
                                                            time_int)

R2: 0.8865525980016313
params: 256
-0.05406275743434473
0.12342107609291415
-0.0026012405083331858
0.07733503207919672
0.14261483138243009
0.15189504417639677
0.1522754403221725
0.010507310040835702
0.06251434560499193
-0.058876151159222956
0.14048681172346544
0.2742587229122199
0.05823274886670722
-0.012630403942770957
-0.13398722476269342
-0.22955753189510064
0.08161392365393773
0.09336178699597468
-0.04260024191823013
0.025494919958711407
-0.025803989674368102
-0.11294247898609587
0.08006120914145716
0.03469388938670688
0.062308517458097915
0.0765561638704896
0.14556905753255922
-0.08531660277455287
-0.0075832804146243465
0.009584565631526334
0.03017869795560575
0.017255943199034965
-0.12306888099655634
-0.05124614508789292
0.00989733707396867
-0.07342851750816752
-0.0690549670160484
-0.0005774181193903966
-0.03841987429876147
-0.07022202749850186
0.0030559301492104757
0.01757369729691345
-0.04852947790868738
-0.006837110884122519
-0.152279151566862
0.07267853406935838
0.02734087664

In [10]:
df_pageviews_norm_results

,topic,low,high,val,pval,std
0,Culture.Biography.Biography*,-0.023301,0.023549,0.000124,False,0.011712
1,Culture.Food and drink,0.043145,0.145139,0.094142,True,0.025498
2,Culture.Internet culture,-0.114048,0.049413,-0.032318,False,0.040865
3,Culture.Linguistics,0.002292,0.129523,0.065907,True,0.031808
4,Culture.Literature,-0.106211,0.031326,-0.037442,False,0.034384
5,Culture.Media.Books,-0.139584,-0.039514,-0.089549,True,0.025018
6,Culture.Media.Entertainment,-0.202911,-0.018555,-0.110733,True,0.046089
7,Culture.Media.Films,-0.266260,-0.101459,-0.183859,True,0.041200
8,Culture.Media.Music,-0.055478,0.042335,-0.006571,False,0.024453
9,Culture.Media.Radio,-0.068298,0.067570,-0.000364,False,0.033967


## Plots image

In [5]:
fig, axes = plt.subplots(1, 3, figsize=(14, 10), sharey=True, sharex=True,
                       gridspec_kw={"hspace": 0.15, "wspace": 0.05})
for idx, topic in enumerate(order_topics):
    for idy, df in enumerate([df_pageviews_results, df_pageviews_norm_results, df_pageviews_norm_results2]):
        row = dict(df[df.topic == topic].iloc[0])
        marker = "." if row["pval"] else "x"

        axes[idy].scatter([row["val"]], [idx], color="black", marker=marker, zorder=2)
        axes[idy].plot([row["low"], row["high"]], [idx, idx], color="#e41a1c", zorder=1)
        axes[idy].axhline(idx+0.5, lw=0.5, ls=":", color='black', alpha=0.5)

    _ = axes[idy].set_yticks(range(0, idx+1))
    _ = axes[idy].set_ylim([-0.5,idx+0.5])
    _ = axes[idy].set_xlim([-0.7, 0.7])

acc = 0
trans = transforms.blended_transform_factory(
     axes[2].transAxes, axes[2].transData)


for x, color, title, in colors_z:
    for idy in range(3):
        axes[idy].fill_between(x=[-0.7, 0.7], y1=idx+1 - acc - 0.5, y2=idx+1 - acc - x - 0.5, color=color, alpha=0.3)
        
    topy = idx - acc
    boty =  idx +1 - acc - x

    plt.plot([1.02, 1.05, 1.05, 1.02], [topy, topy, boty ,boty], transform=trans, clip_on=False, color="gray")
    plt.text(1.06, (topy + boty)/2, title, transform=trans,  color="gray", va="center")
     
    acc += x
    

_ = axes[0].set_yticklabels([v.split(".")[-1].replace("*", "").replace("&", "\&") for v in order_topics])

for ax in axes:
    ax.axvline(0, zorder=0, color="black", ls="-", alpha=0.3)
    
# boxes
vals = [
    (0.23, 0.36),
    (0.36, 0.76),
    (0.23, 0.76),
]

titles = [
    '(a) Before mobility changepoint\nvs.\n After mobility changepoint',
    '(b) After mobility changepoint\nvs.\n After return to baseline',
    '(c) Before mobility changepoint \nvs.\n After return to baseline'
]

for val, tit, ax in zip(vals, titles, axes):
    ax.plot([0.3, 0.3], [1.02, 1.05], clip_on=False, transform=ax.transAxes, color="gray")
    ax.text(0.3, 1.051, "mobility",  transform=ax.transAxes, color="gray",  ha="center", va="bottom")
    ax.plot([0.7, 0.7], [1.02, 1.05], clip_on=False, transform=ax.transAxes, color="gray")
    ax.text(0.7, 1.051, "normality",  transform=ax.transAxes, color="gray",  ha="center", va="bottom")

    ax.text(val[0], 1.035, "pre ",  transform=ax.transAxes, color="gray",  ha="center", va="center", zorder=10,
                     bbox={'facecolor':'white','alpha':1,'edgecolor':'tab:orange','pad':3})
    ax.text(val[1] + 0.01, 1.035, "post",  transform=ax.transAxes, color="gray",  ha="center", va="center", zorder=10,
           bbox={'facecolor':'white','alpha':1,'edgecolor':'tab:purple','pad':3})

    ax.plot([0.15, 0.15, 0.85, 0.85, 0.15], [1.01, 1.07125, 1.07125, 1.01, 1.01], 
            clip_on=False, transform=ax.transAxes, color="black", 
            lw=0.5, ls=":")
    ax.text(0.5, 1.075, tit, transform=ax.transAxes, size=12, ha="center", va="bottom")
    ax.tick_params(axis='y', which='both', labelsize=11)
    ax.set_xlabel("Relative Increase ($\log_e$)", size=14)

set_size(fig, (14, 10))
fig.savefig("../images/topics_master.pdf", bbox_inches="tight")

RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 1400x1000 with 3 Axes>

---